In [ ]:
# Third-party
import astropy.coordinates as coord
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
plt.style.use('apw-notebook')
%matplotlib inline
import h5py

from keras.models import Sequential
from keras.layers import Convolution1D, Convolution2D, GlobalMaxPooling1D, MaxPooling1D
from keras.layers.core import Dense, Activation, Dropout, Flatten

In [ ]:
with h5py.File('../data/testdata.h5', 'r') as f:
    X = f['X'][:].astype(np.float32)
    y = f['y'][:].astype(np.float32)

In [ ]:
X.shape[0]

In [ ]:
n_train = 512
n_valid = 256
n_test = X.shape[0] - n_train - n_valid
print("Train, Valid, Test", n_train, n_valid, n_test)

In [ ]:
X_train = X[:n_train]
y_train = y[:n_train,:1]

X_valid = X[n_train:n_train+n_valid]
y_valid = y[n_train:n_train+n_valid,:1]

X_test = X[n_train+n_valid:]
y_test = y[n_train+n_valid:,:1]

In [ ]:
n_objects, n_pixels = X_train.shape
print(n_pixels)

In [ ]:
n_objects, n_pixels

In [ ]:
model = Sequential()

model.add(Convolution1D(nb_filter=32, filter_length=16, input_dim=1))
model.add(Activation('sigmoid'))
model.add(MaxPooling1D(pool_length=4))

# We add a vanilla hidden layer:
# model.add(Dense(128))
# model.add(Dropout(0.5))

model.add(Dense(24))
model.add(Dropout(0.5))
model.add(Activation('sigmoid'))

model.add(GlobalMaxPooling1D())

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(1))

model.compile(loss="mse", optimizer="rmsprop")

In [ ]:
model.summary()

In [ ]:
model.fit(X_train.reshape(len(X_train), n_pixels, 1), y_train, nb_epoch=64, batch_size=32,
          validation_data=(X_valid.reshape(len(X_valid), n_pixels, 1), y_valid), verbose=True)

In [ ]:
score = model.evaluate(X_test.reshape(len(X_test), n_pixels, 1), y_test, batch_size=16)
score

In [ ]:
y_pred = model.predict(X_test.reshape(len(X_test), n_pixels, 1))
y_pred

In [ ]:
plt.hist(np.log10(np.abs((y_test-y_pred)/y_test)), bins='auto');